In [2]:
import pandas as pd

In [3]:
import sqlite3 as sql

In [4]:
con = sql.connect('C:/Users/marlo/Desktop/UFM/Sexto Semestre/Data Wrangling/Data-Wrangling/Laboratorio # 3/sql-murder-mystery.db')

In [5]:
#Quiero ver que tablas tengo para trabajar
query = "SELECT name FROM sqlite_master WHERE type = 'table'"
tables = pd.read_sql(query,con)
tables

,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


In [6]:
#Quiero Familiarizarme con la tabla de reportes
query2 = "Select *From crime_scene_report"
crime= pd.read_sql(query2,con)
crime

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
...,...,...,...,...
1223,20180430,bribery,\n,Garden Grove
1224,20180430,fraud,‘Why not?’ said the March Hare.\n,Houma
1225,20180430,assault,\n,Fontana
1226,20180501,assault,be NO mistake about it: it was neither more no...,Trenton


In [7]:
#Uso la informacion del caso
query3 = "Select *From crime_scene_report WHERE date = 20180115 AND city = 'SQL City'"
Condiciones= pd.read_sql(query3,con)
Condiciones

,date,type,description,city
0,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot hi...",SQL City
1,20180115,assault,Report Not Found,SQL City
2,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [8]:
#Linea para que me salga todo el codigo
pd.set_option('display.max_colwidth',None)

In [9]:
#reprotes completos
query3 = "Select description From crime_scene_report WHERE date = 20180115 AND city = 'SQL City' "
reportes= pd.read_sql(query3,con)
reportes

,description
0,"Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!"
1,Report Not Found
2,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave""."


In [10]:
#De los 3 reprotes el mas solido era el tercero por el cual lo investigare primero
query4 ="Select * From person Where address_street_name = 'Franklin Ave' And name = 'Annabel Miller'"
Anna = pd.read_sql(query4,con)
Anna

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [11]:
#Quiero saber mas de Annabel Miller
query5 = "Select * From drivers_license Where id = 490173"
AnnaLicense = pd.read_sql(query5,con)
AnnaLicense

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,490173,35,65,green,brown,female,23AM98,Toyota,Yaris


In [13]:
#Entrevista de Annabel
q6 = "Select * From interview Where person_id = 16371"
EAnna = pd.read_sql(q6,con)
EAnna

,person_id,transcript
0,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [19]:
#Registro de gimnasio del dia el cual Annabell habla en su entrevita
q7 = "Select * From get_fit_now_check_in Where check_in_date = 20180109"
gym = pd.read_sql(q7,con)
gym

,membership_id,check_in_date,check_in_time,check_out_time
0,X0643,20180109,957,1164
1,UK1F2,20180109,344,518
2,XTE42,20180109,486,1124
3,1AE2H,20180109,461,944
4,6LSTG,20180109,399,515
5,7MWHJ,20180109,273,885
6,GE5Q8,20180109,367,959
7,48Z7A,20180109,1600,1730
8,48Z55,20180109,1530,1700
9,90081,20180109,1600,1700


In [20]:
#Familiarizarme con la tabla del gimnasio
q8 = "Select * From get_fit_now_member"
gymMem = pd.read_sql(q8,con)
gymMem

,id,person_id,name,membership_start_date,membership_status
0,NL318,65076,Everette Koepke,20170926,gold
1,AOE21,39426,Noe Locascio,20171005,regular
2,2PN28,63823,Jeromy Heitschmidt,20180215,silver
3,0YJ24,80651,Waneta Wellard,20171206,gold
4,3A08L,32858,Mei Bianchin,20170401,silver
...,...,...,...,...,...
179,2V137,41693,Wendell Dulany,20171219,silver
180,4KB72,79110,Emile Hege,20170522,regular
181,48Z7A,28819,Joe Germuska,20160305,gold
182,48Z55,67318,Jeremy Bowers,20160101,gold


In [22]:
# Quiero tener los nombres eh informacion de los que fueron ese dia al gimnasio
q9 = """
    SELECT
            A.person_id
            ,A.name
            ,B.check_in_time
            ,B.check_out_time
            ,B.check_in_date
        FROM get_fit_now_member A
        LEFT JOIN get_fit_now_check_in B ON B.membership_id = A.id
         Where B.check_in_date = 20180109
"""
Sospechosos = pd.read_sql(q9,con)
Sospechosos

,person_id,name,check_in_time,check_out_time,check_in_date
0,15247,Shondra Ledlow,957,1164,20180109
1,28073,Zackary Cabotage,344,518,20180109
2,55662,Sarita Bartosh,486,1124,20180109
3,10815,Adriane Pelligra,461,944,20180109
4,83186,Burton Grippe,399,515,20180109
5,31523,Blossom Crescenzo,273,885,20180109
6,92736,Carmen Dimick,367,959,20180109
7,28819,Joe Germuska,1600,1730,20180109
8,67318,Jeremy Bowers,1530,1700,20180109
9,16371,Annabel Miller,1600,1700,20180109


In [23]:
# Los unicos que estaban en el gym al mismo tiempo que Annabel eran Joe Germuska y Jeremy Bowers


In [24]:
#Entrevista de Joe
q10 = "Select * From interview Where person_id = 28819"
ES1 = pd.read_sql(q10,con)
ES1

,person_id,transcript


In [ ]:
# Joe no tuvo ninguna Entrevista

In [25]:
#Entrevista de Jeremy
q11 = "Select * From interview Where person_id = 67318"
ES2 = pd.read_sql(q11,con)
ES2

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [ ]:
#culpable es Jeremy , ahora hay que averiguar quien es esta mujer.

In [26]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [38]:
#Registro de Facebook condicionado con el evento de la entrevista y el mes
q12 = "Select * From facebook_event_checkin Where event_name ='SQL Symphony Concert' And date > 20171130 And date < 20180101"
M1 = pd.read_sql(q12,con)
M1

,person_id,event_id,event_name,date
0,62596,1143,SQL Symphony Concert,20171225
1,19260,1143,SQL Symphony Concert,20171214
2,58898,1143,SQL Symphony Concert,20171220
3,69699,1143,SQL Symphony Concert,20171214
4,19292,1143,SQL Symphony Concert,20171213
5,43366,1143,SQL Symphony Concert,20171207
6,92343,1143,SQL Symphony Concert,20171212
7,28582,1143,SQL Symphony Concert,20171220
8,28582,1143,SQL Symphony Concert,20171215
9,81526,1143,SQL Symphony Concert,20171202


In [39]:
# Solo hay 2 personas que fueron 3 veces
# Sospechosos id = 99716 y 24556 , Hay que ver tabla de licencia para comparar informacion de entrevista con base de datos

In [40]:
q13 = "Select * From person Where id=99716"
MS1 = pd.read_sql(q13,con)
MS1

,id,name,license_id,address_number,address_street_name,ssn
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [41]:
q14 = "Select * From person Where id=24556"
MS2 = pd.read_sql(q14,con)
MS2

,id,name,license_id,address_number,address_street_name,ssn
0,24556,Bryan Pardo,101191,703,Machine Ln,816663882


In [ ]:
# como solo una de las personas es mujer puedo estar seguro sin revisar su aspecto en la base de datos de licencia 
# Por lo cual se que ella es la culpable

In [42]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Miranda Priestly')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'

In [ ]:
# Rajo al champagne pero me pico por una miche.